# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
I gone through several sites and came to the conclusion
- `batch_size` - For a good batch size, It can be anywhere between 32 - 256. Depend on the  number of training data we have.
- `vocab_threshold` - I started with 5 since it seems plausible solution.
- `embed_size` - I chose 512 value based on Paper 1
- `num_epochs` - Sicne the model will take too much time to one epoch, i left to 3 and watched the changes in the training loss to determine the performance.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
- The provided value worked pretty well for the starting point. That's why i didn't amend the values

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
- Since we are using ResNet for CNN model we don't need to train the whole model. We just need to train the last layer.
- As for the RNN we need to train the whole model. That's why all the parameters were set to learnable.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 
I refered several paper to choose the optimizer. Most of them used ADAM optimizer since its robust. 

In [4]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())  

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 423/414113 [00:00<01:37, 4229.82it/s]

  0%|          | 890/414113 [00:00<01:34, 4351.50it/s]

  0%|          | 1372/414113 [00:00<01:32, 4479.53it/s]

  0%|          | 1847/414113 [00:00<01:30, 4554.67it/s]

  1%|          | 2309/414113 [00:00<01:30, 4571.21it/s]

  1%|          | 2776/414113 [00:00<01:29, 4599.62it/s]

  1%|          | 3187/414113 [00:00<01:33, 4418.21it/s]

  1%|          | 3655/414113 [00:00<01:31, 4492.33it/s]

  1%|          | 4119/414113 [00:00<01:30, 4533.15it/s]

  1%|          | 4596/414113 [00:01<01:29, 4600.07it/s]

  1%|          | 5069/414113 [00:01<01:28, 4636.80it/s]

  1%|▏         | 5545/414113 [00:01<01:27, 4672.16it/s]

  1%|▏         | 6008/414113 [00:01<01:30, 4508.86it/s]

  2%|▏         | 6465/414113 [00:01<01:30, 4524.60it/s]

  2%|▏         | 6930/414113 [00:01<01:29, 4559.47it/s]

  2%|▏         | 7385/414113 [00:01<01:29, 4540.64it/s]

  2%|▏         | 7859/414113 [00:01<01:28, 4

 15%|█▍        | 60436/414113 [00:13<01:18, 4516.40it/s]

 15%|█▍        | 60924/414113 [00:13<01:16, 4618.82it/s]

 15%|█▍        | 61397/414113 [00:13<01:15, 4646.67it/s]

 15%|█▍        | 61870/414113 [00:13<01:15, 4636.94it/s]

 15%|█▌        | 62339/414113 [00:13<01:16, 4608.53it/s]

 15%|█▌        | 62816/414113 [00:13<01:15, 4655.49it/s]

 15%|█▌        | 63292/414113 [00:13<01:14, 4685.31it/s]

 15%|█▌        | 63772/414113 [00:13<01:14, 4718.32it/s]

 16%|█▌        | 64246/414113 [00:13<01:14, 4705.27it/s]

 16%|█▌        | 64718/414113 [00:14<01:14, 4686.26it/s]

 16%|█▌        | 65197/414113 [00:14<01:14, 4714.99it/s]

 16%|█▌        | 65673/414113 [00:14<01:13, 4727.57it/s]

 16%|█▌        | 66155/414113 [00:14<01:13, 4753.50it/s]

 16%|█▌        | 66635/414113 [00:14<01:12, 4764.99it/s]

 16%|█▌        | 67112/414113 [00:14<01:13, 4738.01it/s]

 16%|█▋        | 67586/414113 [00:14<01:13, 4721.09it/s]

 16%|█▋        | 68061/414113 [00:14<01:13, 4728.03it/s]

 17%|█▋       

 29%|██▉       | 120987/414113 [00:26<01:01, 4771.91it/s]

 29%|██▉       | 121468/414113 [00:26<01:01, 4782.18it/s]

 29%|██▉       | 121947/414113 [00:26<01:01, 4757.53it/s]

 30%|██▉       | 122423/414113 [00:26<01:01, 4730.93it/s]

 30%|██▉       | 122897/414113 [00:26<01:01, 4703.83it/s]

 30%|██▉       | 123368/414113 [00:26<01:01, 4695.42it/s]

 30%|██▉       | 123839/414113 [00:26<01:01, 4697.17it/s]

 30%|███       | 124315/414113 [00:26<01:01, 4714.58it/s]

 30%|███       | 124787/414113 [00:26<01:01, 4701.46it/s]

 30%|███       | 125261/414113 [00:26<01:01, 4711.51it/s]

 30%|███       | 125733/414113 [00:27<01:01, 4706.96it/s]

 30%|███       | 126206/414113 [00:27<01:01, 4713.12it/s]

 31%|███       | 126686/414113 [00:27<01:00, 4738.50it/s]

 31%|███       | 127162/414113 [00:27<01:00, 4742.76it/s]

 31%|███       | 127637/414113 [00:27<01:00, 4731.81it/s]

 31%|███       | 128111/414113 [00:27<01:00, 4699.01it/s]

 31%|███       | 128584/414113 [00:27<01:00, 4705.52it/s

 44%|████▎     | 181101/414113 [00:39<00:49, 4679.19it/s]

 44%|████▍     | 181582/414113 [00:39<00:49, 4715.27it/s]

 44%|████▍     | 182055/414113 [00:39<00:49, 4694.94it/s]

 44%|████▍     | 182536/414113 [00:39<00:48, 4726.19it/s]

 44%|████▍     | 183010/414113 [00:39<00:49, 4713.94it/s]

 44%|████▍     | 183482/414113 [00:39<00:49, 4646.07it/s]

 44%|████▍     | 183948/414113 [00:39<00:49, 4646.46it/s]

 45%|████▍     | 184421/414113 [00:39<00:49, 4670.81it/s]

 45%|████▍     | 184896/414113 [00:39<00:48, 4693.02it/s]

 45%|████▍     | 185378/414113 [00:39<00:48, 4728.17it/s]

 45%|████▍     | 185856/414113 [00:40<00:48, 4741.08it/s]

 45%|████▍     | 186331/414113 [00:40<00:48, 4701.21it/s]

 45%|████▌     | 186821/414113 [00:40<00:47, 4758.53it/s]

 45%|████▌     | 187298/414113 [00:40<00:47, 4748.16it/s]

 45%|████▌     | 187774/414113 [00:40<00:47, 4740.88it/s]

 45%|████▌     | 188249/414113 [00:40<00:47, 4724.55it/s]

 46%|████▌     | 188722/414113 [00:40<00:48, 4687.73it/s

 58%|█████▊    | 241278/414113 [00:51<00:36, 4680.50it/s]

 58%|█████▊    | 241754/414113 [00:51<00:36, 4700.96it/s]

 58%|█████▊    | 242225/414113 [00:51<00:36, 4674.50it/s]

 59%|█████▊    | 242707/414113 [00:52<00:36, 4715.57it/s]

 59%|█████▊    | 243182/414113 [00:52<00:36, 4725.53it/s]

 59%|█████▉    | 243666/414113 [00:52<00:35, 4759.18it/s]

 59%|█████▉    | 244143/414113 [00:52<00:35, 4742.52it/s]

 59%|█████▉    | 244618/414113 [00:52<00:35, 4721.37it/s]

 59%|█████▉    | 245097/414113 [00:52<00:35, 4739.92it/s]

 59%|█████▉    | 245581/414113 [00:52<00:35, 4767.21it/s]

 59%|█████▉    | 246061/414113 [00:52<00:35, 4774.40it/s]

 60%|█████▉    | 246539/414113 [00:52<00:35, 4770.63it/s]

 60%|█████▉    | 247017/414113 [00:53<00:35, 4730.47it/s]

 60%|█████▉    | 247493/414113 [00:53<00:35, 4737.29it/s]

 60%|█████▉    | 247967/414113 [00:53<00:35, 4730.84it/s]

 60%|█████▉    | 248441/414113 [00:53<00:35, 4698.02it/s]

 60%|██████    | 248911/414113 [00:53<00:35, 4655.59it/s

 73%|███████▎  | 301401/414113 [01:04<00:23, 4703.61it/s]

 73%|███████▎  | 301889/414113 [01:04<00:23, 4755.07it/s]

 73%|███████▎  | 302365/414113 [01:04<00:23, 4751.90it/s]

 73%|███████▎  | 302841/414113 [01:04<00:23, 4748.35it/s]

 73%|███████▎  | 303327/414113 [01:04<00:23, 4781.00it/s]

 73%|███████▎  | 303806/414113 [01:05<00:23, 4780.02it/s]

 73%|███████▎  | 304285/414113 [01:05<00:23, 4760.19it/s]

 74%|███████▎  | 304762/414113 [01:05<00:22, 4755.31it/s]

 74%|███████▎  | 305238/414113 [01:05<00:23, 4661.24it/s]

 74%|███████▍  | 305710/414113 [01:05<00:23, 4676.92it/s]

 74%|███████▍  | 306181/414113 [01:05<00:23, 4685.30it/s]

 74%|███████▍  | 306651/414113 [01:05<00:22, 4687.86it/s]

 74%|███████▍  | 307120/414113 [01:05<00:22, 4688.25it/s]

 74%|███████▍  | 307600/414113 [01:05<00:22, 4718.74it/s]

 74%|███████▍  | 308073/414113 [01:05<00:22, 4714.89it/s]

 75%|███████▍  | 308549/414113 [01:06<00:22, 4725.99it/s]

 75%|███████▍  | 309024/414113 [01:06<00:22, 4732.51it/s

 87%|████████▋ | 361249/414113 [01:17<00:11, 4698.64it/s]

 87%|████████▋ | 361728/414113 [01:17<00:11, 4724.47it/s]

 87%|████████▋ | 362201/414113 [01:17<00:11, 4718.35it/s]

 88%|████████▊ | 362673/414113 [01:17<00:10, 4708.13it/s]

 88%|████████▊ | 363144/414113 [01:18<00:10, 4680.92it/s]

 88%|████████▊ | 363617/414113 [01:18<00:10, 4692.83it/s]

 88%|████████▊ | 364087/414113 [01:18<00:10, 4690.04it/s]

 88%|████████▊ | 364560/414113 [01:18<00:10, 4701.32it/s]

 88%|████████▊ | 365035/414113 [01:18<00:10, 4715.48it/s]

 88%|████████▊ | 365511/414113 [01:18<00:10, 4726.88it/s]

 88%|████████▊ | 365984/414113 [01:18<00:10, 4684.97it/s]

 88%|████████▊ | 366461/414113 [01:18<00:10, 4708.22it/s]

 89%|████████▊ | 366932/414113 [01:18<00:10, 4705.04it/s]

 89%|████████▊ | 367406/414113 [01:18<00:09, 4715.27it/s]

 89%|████████▉ | 367878/414113 [01:19<00:09, 4701.26it/s]

 89%|████████▉ | 368358/414113 [01:19<00:09, 4728.71it/s]

 89%|████████▉ | 368831/414113 [01:19<00:09, 4725.07it/s

Done (t=0.89s)
creating index...
index created!
Obtaining caption lengths...


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [5]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/6471], Loss: 4.5481, Perplexity: 94.44878
Epoch [1/3], Step [200/6471], Loss: 4.3511, Perplexity: 77.56660
Epoch [1/3], Step [300/6471], Loss: 3.4070, Perplexity: 30.17447
Epoch [1/3], Step [400/6471], Loss: 3.3101, Perplexity: 27.38847
Epoch [1/3], Step [500/6471], Loss: 3.3274, Perplexity: 27.8646
Epoch [1/3], Step [600/6471], Loss: 3.0565, Perplexity: 21.25279
Epoch [1/3], Step [700/6471], Loss: 3.1221, Perplexity: 22.6942
Epoch [1/3], Step [800/6471], Loss: 3.1887, Perplexity: 24.25605
Epoch [1/3], Step [900/6471], Loss: 3.3112, Perplexity: 27.4174
Epoch [1/3], Step [1000/6471], Loss: 2.9058, Perplexity: 18.2805
Epoch [1/3], Step [1100/6471], Loss: 3.1403, Perplexity: 23.1101
Epoch [1/3], Step [1200/6471], Loss: 3.2628, Perplexity: 26.1230
Epoch [1/3], Step [1300/6471], Loss: 2.6272, Perplexity: 13.8345
Epoch [1/3], Step [1400/6471], Loss: 2.6457, Perplexity: 14.0929
Epoch [1/3], Step [1500/6471], Loss: 2.6806, Perplexity: 14.5945
Epoch [1/3], Step [1600/6471

Epoch [2/3], Step [6300/6471], Loss: 1.9999, Perplexity: 7.38823
Epoch [2/3], Step [6400/6471], Loss: 2.1639, Perplexity: 8.70470
Epoch [3/3], Step [100/6471], Loss: 1.8922, Perplexity: 6.633942
Epoch [3/3], Step [200/6471], Loss: 2.1977, Perplexity: 9.00406
Epoch [3/3], Step [300/6471], Loss: 1.8214, Perplexity: 6.18073
Epoch [3/3], Step [400/6471], Loss: 2.1810, Perplexity: 8.85505
Epoch [3/3], Step [500/6471], Loss: 1.9005, Perplexity: 6.689404
Epoch [3/3], Step [600/6471], Loss: 1.9341, Perplexity: 6.91802
Epoch [3/3], Step [700/6471], Loss: 2.1635, Perplexity: 8.70165
Epoch [3/3], Step [800/6471], Loss: 1.9005, Perplexity: 6.68925
Epoch [3/3], Step [900/6471], Loss: 1.8517, Perplexity: 6.37089
Epoch [3/3], Step [1000/6471], Loss: 2.0760, Perplexity: 7.9726
Epoch [3/3], Step [1100/6471], Loss: 1.9497, Perplexity: 7.02687
Epoch [3/3], Step [1200/6471], Loss: 1.8424, Perplexity: 6.31179
Epoch [3/3], Step [1300/6471], Loss: 2.2143, Perplexity: 9.15545
Epoch [3/3], Step [1400/6471], Lo

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.